**Mangrove monitoring using multitemporal Sentinel-2 SR and Mangrove Vegetation Index**

In [ ]:
"""
Mangrove Vegetation Index (MVI) citation:

Baloloy, A.B., Blanco, A.C., Ana, R.R.C.S., Nadaoka, K., 2020,
Development and application of a new mangrove vegetation index (MVI) for rapid and accurate mangrove mapping,
ISPRS Journal of Photogrammetry and Remote Sensing, Vol. 166, pp. 95-117.
https://doi.org/10.1016/j.isprsjprs.2020.06.001
"""

In [ ]:
# Code provided by Syam'ani (https://github.com/syamaniulm)

In [ ]:
import ee

In [ ]:
ee.Authenticate()

In [ ]:
# Initialize your Earth Engine cloud project

ee.Initialize(project='ee-yourproject')

In [ ]:
# Access google drive and open a shapefile

import geopandas as gpd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Replace 'path/to/your/shapefile.shp' with the actual path to your shapefile in Google Drive
shapefile_path = '/content/drive/My Drive/Your_folder/Your_shapefile.shp'

# Open shapefile using Geopandas
try:
    gdf = gpd.read_file(shapefile_path)
    gdf.plot()
except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Converting shapefile into Earth Engine geometry

import json

region_js = json.loads(gdf.to_json())
region_fc = ee.FeatureCollection(region_js)
region = ee.Geometry(region_fc.geometry())

In [ ]:
# Region center

region_center = region.centroid().coordinates().getInfo()
print(region_center)

In [ ]:
# Determining parameters

num_years = 6     # years, adjust to your needs
step = 1          # year(s), adjust to your needs
last_year = 2024  # adjust to your needs

start_date = []
end_date = []
years = []

for i in range(num_years):
    year = last_year - num_years + i + step
    start_date.append(f'{year}-01-01')
    end_date.append(f'{year}-12-31')
    years.append(year)

print(start_date)
print(end_date)
print(years)

In [ ]:
# Access multitempiral Sentinel-2 surface reflectance

s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

s2_image_list = []

for i in range(num_years):
  dataset = s2_col.filterBounds(region) \
      .filterDate(start_date[i], end_date[i]) \
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  s2_image_list.append(dataset.median())

In [ ]:
# Calculating multitemporal Mangrove Vegetation Index (MVI)
# Creating mangrove extent image

mvi_list = []
mgrv_list = []

for i in range(num_years):
  green = s2_image_list[i].select('B3')
  nir = s2_image_list[i].select('B8')
  swir1 = s2_image_list[i].select('B11')
  mvi = (nir.subtract(green)).divide(swir1.subtract(green)).rename('MVI')
  mask = mvi.gte(4.5).And(mvi.lte(16.5)).rename('Mangrove')
  mvi = mvi.updateMask(mask)
  mvi = mvi.clip(region)
  mvi_list.append(mvi)
  mgrv_list.append(mask)

In [ ]:
# Create geemap MVI timeseries

import geemap

# Create a list of dates corresponding to the MVI images
dates = [f'{year}' for year in years]  # Or use actual dates if available

# Visualization parameters
vis_params = {
    'min': 4.5,
    'max': 16.5,
    'palette': ['seagreen', 'darkgreen', 'darkblue']
}

# Create the MVI time series visualization using geemap
Map = geemap.Map(basemap='Esri.WorldTopoMap')
Map.centerObject(region, 11)

# Create image collection from image list
mvi_col = ee.ImageCollection.fromImages(mvi_list)

# Create labels
labels = [str(year) for year in years]

# Add a layer control panel
Map.add_time_slider(mvi_col, vis_params, labels=labels, time_interval=1)

# Display the map
Map

In [ ]:
# Mangrove area calculation

for i in range(num_years):
  area = mgrv_list[i].reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=region,
      scale=10,
      maxPixels=1e13
  ).get('Mangrove').getInfo()
  print(f'Mangrove area in {years[i]}: {round(area/100,2)} hectares')